<a href="https://www.kaggle.com/code/isabelawhitaker/choropleth?scriptVersionId=174179029" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
       

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#import packages and data
import pandas as pd
df = pd.read_csv(r'/kaggle/input/uk-housing-prices-paid/price_paid_records.csv')

import matplotlib.pyplot as plt

import seaborn as sns
sns.set(palette='deep')

import numpy as np

from matplotlib.axes._axes import _log as matplotlib_axes_logger
matplotlib_axes_logger.setLevel('ERROR')

import json
import plotly.express as px
import geopandas
import pandas
import matplotlib.pyplot as plt
import contextily as ctx

In [ ]:
#Make a copy to keep the original df intact in case it is required later on
df1=df.copy()

In [ ]:
#Drop columns 1, 5 and 10 as they are not relevant this time
columns_to_drop= ['Transaction unique identifier', 'Duration', 'Record Status - monthly file only']
df1=df1.drop(columns=columns_to_drop)

#
df1 = df1[df1['Property Type'] != 'O']

In [ ]:
#Convert the 'Date of transfer column to a pandas datetime type for easier'
df1['Date of Transfer'] = pd.to_datetime(df1['Date of Transfer'])
df1.info()

In [ ]:
#Create a numerical column for year of transfer
df1['Year of Transfer'] = pd.to_datetime(df1['Date of Transfer']).dt.year
#Create a numerical column for month of transfer
df1['Month of Transfer'] = pd.to_datetime(df1['Date of Transfer']).dt.month

In [43]:
#Define a function to assign each County to one of the 9 administrative Regions of England or (can easily be updated) Wales

def find_region(x):
    
    if x == 'CORNWALL' or x=='SOMERSET' or x== 'AVON' or x=='BOURNEMOUTH' or x=='GLOUCESTERSHIRE' or x=='DORSET' or x== 'DEVON' or x== 'ISLES OF SCILLY' or x=='SOUTH GLOUCESTERSHIRE' or x=='NORTH SOMERSET' or x=='CITY OF BRISTOL' or x=='TORBAY' or x=='BATH AND NORTH EAST SOMERSET' or x=='POOLE' or x=='THAMESDOWN'or x== 'WILTSHIRE' or x== 'CITY OF PLYMOUTH' or x=='SWINDON':
        return 'South West'
    
    elif x == 'WEST MIDLANDS' or  x=='STOKE-ON-TRENT' or x=='SHROPSHIRE' or x=='HEREFORDSHIRE'or x=='HEREFORD AND WORCESTER' or x=='WORCESTERSHIRE' or x=='STAFFORDSHIRE' or x=='WREKIN' or x=='WARWICKSHIRE':
        return 'West Midlands'
    
    elif x == 'GREATER LONDON':
        return 'London'
    
    elif x == 'KENT' or x=='ISLE OF WIGHT' or x=='PORTSMOUTH' or x=='MEDWAY' or x=='OXFORDSHIRE' or x=='SOUTHAMPTON' or x=='WEST BERKSHIRE' or x=='BRIGHTON AND HOVE' or x=='BERKSHIRE' or x=='SLOUGH' or x=='HAMPSHIRE' or x=='WOKINGHAM' or x=='READING' or x=='WINDSOR AND MAIDENHEAD' or x== 'BRACKNELL FOREST' or x=='MILTON KEYNES' or x=='SURREY' or x=='BUCKINGHAMSHIRE' or x=='WEST SUSSEX' or x=='EAST SUSSEX':
        return 'South East'
    
    elif x == 'YORK' or x=='NORTH YORKSHIRE' or x=='WEST YORKSHIRE' or x=='SOUTH YORKSHIRE' or x=='EAST RIDING OF YORKSHIRE' or x== 'NORTH EAST LINCOLNSHIRE' or x=='CITY OF KINGSTON UPON HULL'or x=='NORTH LINCOLNSHIRE' or x=='NORTH EAST LINCOLNSHIRE' or x=='HUMBERSIDE':
        return 'Yorkshire and the Humber'
    
    elif x == 'CUMBRIA' or x=='BLACKPOOL' or x=='GREATER MANCHESTER' or x=='MERSEYSIDE' or x=='LANCASHIRE' or x=='WARRINGTON' or x=='CHESHIRE' or x=='HALTON' or x=='CHESHIRE WEST AND CHESTER' or x=='BLACKBURN WITH DARWEN' or x=='CHESHIRE EAST':
        return 'North West'

    elif x == 'TYNE AND WEAR' or x=='DARLINGTON' or x=='HARTLEPOOL' or x=='COUNTY DURHAM' or x=='NORTHUMBERLAND' or x=='DURHAM' or x=='REDCAR AND CLEVELAND' or x=='STOCKTON-ON-TEES' or x=='MIDDLESBROUGH' or x=='CLEVELAND':
        return 'North East'
    
    elif x == 'NORFOLK' or x=='THURROCK' or x=='BEDFORDSHIRE' or x=='CITY OF PETERBOROUGH'or x=='CAMBRIDGESHIRE' or x=='LUTON' or x=='CENTRAL BEDFORDSHIRE' or x=='BEDFORD'or x=='SOUTHEND-ON-SEA' or x=='ESSEX' or x=='NORTHAMPTONSHIRE' or x=='HERTFORDSHIRE'or x=='NORFOLK' or x=='SUFFOLK':
        return 'East of England'
    
    elif x == 'DERBYSHIRE' or x=='LEICESTER' or x=='RUTLAND' or x=='LINCOLNSHIRE' or x=='CITY OF NOTTINGHAM' or x=='CITY OF DERBY' or x=='NOTTINGHAMSHIRE' or x=='LEICESTERSHIRE':
        return 'East Midlands'
    
    else:
        return 'Wales'
   
#Apply this function to the County column, creating a new column called Region'
df1['Region'] =df1['County'].apply(find_region)

In [ ]:
df_eng= df1[df1['Region'] != "Wales"]
df_eng.head()

In [44]:
geojson = geopandas.read_file(GEOJSON_FILE)